In [ ]:
# MNIST Example.
#
# Adapted from: https://github.com/pytorch/examples/blob/master/mnist/main.py
#  Commit hash: 0f0c9131ca5c79d1332dce1f4c06fe942fbdc665

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms

In [ ]:
# Model definition.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
# Prepare data.

# Download datasets.
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset_train = datasets.MNIST('./', train=True,  transform=transform, download=True)
dataset_test  = datasets.MNIST('./', train=False, transform=transform)

# Create Dataloaders.
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=64)
dataloader_test  = torch.utils.data.DataLoader(dataset_test , batch_size=1000)

In [ ]:
# Training - Testing.

from neuron_models.framework import Trainer, Tester
from tqdm import trange

params = {
    'epochs': 14,
    'lr': 1,
    'loss_fn': F.nll_loss,
    'optimizer': torch.optim.Adadelta,
}

model = Net()

trainer = Trainer(model, dataloader_train, dtype=torch.float32)
tester  = Tester(model, dataloader_test, dtype=torch.float32)

LOSS_TRAIN = np.array([])
LOSS_TEST  = np.array([])
ACCURACY   = np.array([])

# Main loop
for epoch in trange(params['epochs']):
    # Train and test.
    model, loss_train      = trainer.run(**params)
    loss_test, accuracy, _ = tester.run(**params)

    # Collect results.
    LOSS_TRAIN = np.append(LOSS_TRAIN, loss_train)
    LOSS_TEST  = np.append(LOSS_TEST, loss_test)
    ACCURACY   = np.append(ACCURACY, accuracy)

In [ ]:
# Figure.

fig = plt.figure(figsize=(18,4))

plt.subplot(1,2,1)
plt.title('Training Loss')
plt.xlabel('epoch*training point')
plt.ylabel('train loss')
for i in range(params['epochs']):
    plt.vlines(
        x = i*len(LOSS_TRAIN)/params['epochs'],
        ymin = 0,
        ymax = 1,
        colors = 'r',
        linestyles=':'
    )
plt.plot(LOSS_TRAIN)

plt.subplot(1,2,2)
plt.title('Testing Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid()
plt.xticks(range(1, len(ACCURACY)+1))
plt.plot(range(1,len(ACCURACY)+1), ACCURACY)

plt.show()